In [168]:
import numpy as np
import os
import pandas as pd
import csv
import re
from collections import Counter
import itertools 
  


In [210]:
def load_data(fPath):
    data = []
    maxNumber = 0
    with open(fPath, newline='') as csvfile:
        dataFile = csv.reader(csvfile, delimiter=',', skipinitialspace=True)
        for row in dataFile:
            curMax = int(max(row))
            maxNumber = curMax if curMax > maxNumber else maxNumber
            data.append([int(i) for i in row])
            # Convert strings to ints

    return data, maxNumber

def load_params(fPath):
    params = []
    with open(fPath, newline='') as csvfile:
        dataFile = csv.reader(csvfile, delimiter=',', skipinitialspace=True)
        # Extract data in tuples. Split line delimited by '=' then find if the left term has a number key or refers to rest or SDC terms.
        for row in dataFile:
            key, val = row[0].split('=')            
            key = re.findall(r'\d+', key)
            key = int(key[0]) if key  != [] else ('rest' if 'rest' in row[0] else 'SDC')           
            params.append((key,float(val)))
    print(params)
    # format params into a sorted array
    return params

def create_M(data, rawParams):
    uniqueItems = []
    # Concat all list in a big one, so we can find the distinct items
    for row in data:
        uniqueItems += row
    uniqueItems = np.sort(np.array(np.unique(uniqueItems), dtype=int))
    #print(uniqueItems)
    # Initialize param file. Column 0 has the items, columns 1 the MIS stated in 'rest' for init
    params = np.ones((2,len(uniqueItems)))
    params[0,:] = uniqueItems; params[1,:] = rawParams[-2][1]
    #print(params, rawParams[-2][1])
    # Find which items are given in the raw Params file. Last 2 lines are always rest and SDC
    for i in rawParams[:-2]:
        item, sup = i[0],i[1]
        params[1, np.where(params[0,:] == item)] = sup 
    # Now sort the params array to create M
    indices = np.argsort(params[1,:])
    M = np.array([params[0, indices], params[1,indices]])
    # Add the original indices at each col for future ease of use. So item with id 4 has its data lying in M(0, M[2,4]) for value and M (1, M[2,4]) for mis(4).
    indices = indices.reshape((1,-1))
    print(M.shape, indices.shape, uniqueItems.shape)
    origIdxs = np.zeros(uniqueItems.shape[0])
    for i, item in enumerate(M[0,:]):
        for j, origItem in enumerate(uniqueItems):
            #print(item, origItem)
            if item == origItem:
                origIdxs[i] = j
                break
    # Create a zip object from two lists
    zipbObj = zip(M[0,:], M[1,:])
    # Create a dictionary from zip object
    H = dict(zipbObj) 
    print("Params: ", params)
    print(uniqueItems, indices[0], origIdxs)
    print('H: ', H)
    print('M: ', M)
    return M, H, rawParams[-1][1]

dataPath = "dummyData1.txt"
paramsPath = "dummyParams1.txt"
T, maxNumber = load_data(dataPath)
rawParams = load_params(paramsPath)
M, H, SDC = create_M(T, rawParams)

[(1, 0.5), (3, 0.25), (2, 0.4), ('rest', 0.2), ('SDC', 0.003)]
(2, 14) (1, 14) (14,)
Params:  [[ 1.    2.    3.    4.    5.    6.    7.    8.    9.   10.   20.   30.
  40.   50.  ]
 [ 0.5   0.4   0.25  0.2   0.2   0.2   0.2   0.2   0.2   0.2   0.2   0.2
   0.2   0.2 ]]
[ 1  2  3  4  5  6  7  8  9 10 20 30 40 50] [ 3  4  5  6  7  8  9 10 11 12 13  2  1  0] [ 3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.  2.  1.  0.]
H:  {4.0: 0.2, 5.0: 0.2, 6.0: 0.2, 7.0: 0.2, 8.0: 0.2, 9.0: 0.2, 10.0: 0.2, 20.0: 0.2, 30.0: 0.2, 40.0: 0.2, 50.0: 0.2, 3.0: 0.25, 2.0: 0.4, 1.0: 0.5}
M:  [[ 4.    5.    6.    7.    8.    9.   10.   20.   30.   40.   50.    3.
   2.    1.  ]
 [ 0.2   0.2   0.2   0.2   0.2   0.2   0.2   0.2   0.2   0.2   0.2   0.25
   0.4   0.5 ]]


In [211]:
def findsubsets(s, n): 
    """ DESCRIPTION: Find all subsets of given length n.
        ARGUMETNS: s (List): List of the original set
                   n (int):  Target subsets size. 
    """
    return list(itertools.combinations(s, n)) 

def init_pass(M,T, debug = False):
    """ DESCRIPTION: Perform the init-pass step as decribed in the paper. Go through the transactions file and find actual support for
                     each item. Then check if that support
        ARGUMETNS: M (matrix): matrix holding the rules in as ascending value
                   T (matrix): matrix holding the transactions. 
    """
    
    numOfTransactions = T.shape[0]
    supMatrix = []
    F = []
    if debug: print(T, M.shape)
    # Step 1. Compute actual level 1 support for all items.
    for i, mis in enumerate(M[1,:]):
        sup, count = 0, 0
        for j, t in enumerate(T):
            if M[0,i] in t:
                count +=1
        sup  =  count / numOfTransactions
        supMatrix.append([M[0,i], sup])
    supMatrix = np.array(supMatrix).transpose()
    if debug: print(supMatrix)
    
    # Now sort the support array so we can start searching in M to create level 2 seed
    indices = np.argsort(supMatrix[1,:])
    supMatrix = np.array([supMatrix[0, indices], supMatrix[1,indices]])
    #print(supMatrix)
    
    # Step 2. Insert first item in M that has valid support. THen each subsequent item j ust havej.count >= mis(i)
    entryMis = 0
    for i, mis in enumerate(M[1,:]):
        # If no item i has been yet pushed into F use this items mis as target; else use mis(i)
        mis = mis if entryMis == 0 else entryMis
        #print(M[0,i])
        # Find candidate's index in support matrix
        index = np.where(supMatrix[0,:] == M[0,i])[0][0]
        #print(index, mis)
        if supMatrix[1, index] >= mis:
            F.append(supMatrix[:,index])
            entryMis = mis if entryMis == 0 else entryMis
        else:
            if debug: print("Rejecting ", supMatrix[0,index], supMatrix[1, index], mis)
    F = np.array(F).transpose()
    return F

def compute_L1(entries, M):
    """ DESCRIPTION: THis funtion compute L matrix where each item i given as input is checked against its mis support. Those items i that
                     have current support <  mis(i) are eliminated. 
        ARGUMENTS: entries (array): Array of candidate entries for current level L array.
                   M (matrix):    Matrix holding all the items, their mis
                                  row 1: Id of entry
                                  row 2: MIS of entry, in ascending order
        RETURNS: L (array)                          
    """
    L = []
    for i, e in enumerate(entries[0,:]):
        idx = np.where(M[0,:] == e)[0][0]
        if entries[1,i] >= M[1,idx]:
            L.append(e)
    return L

def level2_candidate_gen(F, M):
    """ DESCRIPTION: THis funtion computes the candidates for C2 as described in the paper
        ARGUMENTS: F (): The matrix F generated from the previous step
        RETURNS: C (List)                          
    """
    C2 = []
    for i, f in enumerate(F[0,:-1]):
        if F[1,i] >= M[f]:
            for j, h in enumerate(F[0, i+1:]):
                if F[1,j] >= M[h]:
                    C2.append([int(f),int(h)])
    return C2
def candidate_generate(L, M):
    """ DESCRIPTION: THis funtion computes the candidates for CK as describe in paper. 
        ARGUMENTS: L (list): List containing L_k-1 sets. They are assumed to be sort in ascending order, both within the itemset
                             and in collection level.
                   M (dict): Dictionary holding all the MIS ruls for each item. Dictionaries have a built in hash search function
                             making search and insert quite fast.  
        RETURNS: C (List):  List of items containing the pruned C_k level candidates.   
    """
    C = []
    # Join step
    # For all items p in L compare with all following items q in L
    for i, p in enumerate(L[:-1]):
        for j, q in enumerate(L):
            # Only join items that are itentical at k-1 steps and differ at last
            if p[:-1] == q[:-1]:
                if p[-1] < q[-1]:
                    # Join items in sorted order
                    C.append(p[:-1]+[p[-1]]+[q[-1]])
    # Prune Step
    print("Length of candidate: ",len(C))
    n = len(C[0])
    Ck = []
    for i, c in enumerate(C):
        subsets = list(itertools.combinations(c, n)) 
        for j, s in enumerate(subsets):
            if (c[1] in s) or (M[c[2]] == M[c[1]]):
                if s not in L:
                    C.pop(i) # delete c in Ck
                
    return C

F = init_pass(M,np.array(T), debug = True)
L1 = compute_L1(F,M)
C2 = level2_candidate_gen(F, H)
C = candidate_generate(C2, H)
print("F: ", F , "\n")
print("L: ", L1, "\n")
print("C2: ", C2 , "\n")
print("C: ", C , "\n")
len(C)

[list([10, 50, 30, 40]) list([1, 2, 4, 6, 8, 9, 10]) list([3, 4, 5, 1])
 list([7, 9, 20, 50]) list([1, 3, 50])] (2, 14)
[[ 4.   5.   6.   7.   8.   9.  10.  20.  30.  40.  50.   3.   2.   1. ]
 [ 0.4  0.2  0.2  0.2  0.2  0.4  0.4  0.2  0.2  0.2  0.6  0.4  0.2  0.6]]
Length of candidate:  224
F:  [[ 4.   5.   6.   7.   8.   9.  10.  20.  30.  40.  50.   3.   2.   1. ]
 [ 0.4  0.2  0.2  0.2  0.2  0.4  0.4  0.2  0.2  0.2  0.6  0.4  0.2  0.6]] 

L:  [4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20.0, 30.0, 40.0, 50.0, 3.0, 1.0] 

C2:  [[4, 5], [4, 6], [4, 7], [4, 8], [4, 9], [4, 10], [4, 20], [4, 30], [4, 40], [4, 50], [4, 3], [4, 2], [5, 6], [5, 7], [5, 8], [5, 9], [5, 10], [5, 20], [5, 30], [5, 40], [5, 50], [5, 2], [6, 7], [6, 8], [6, 9], [6, 10], [6, 20], [6, 30], [6, 40], [6, 50], [6, 1], [7, 8], [7, 9], [7, 10], [7, 20], [7, 30], [7, 40], [7, 50], [8, 9], [8, 10], [8, 20], [8, 30], [8, 40], [8, 50], [8, 3], [9, 10], [9, 20], [9, 30], [9, 40], [9, 50], [9, 3], [9, 2], [10, 20], [10, 30], [10, 

112

In [262]:
# MS-Apriori
numOfTransactions = len(T)
Lk = []
F = init_pass(M,np.array(T))
L1 = compute_L1(F,M)
k = 2
Lk_1 = L1
# Line 4: Start of main loop
while len(Lk_1) > 1:
    print("Iteration: ", k)
    if k == 2:
        C = level2_candidate_gen(F, H)
    else:
        C = candidate_generate(Lk_1, H)
        
    ckCounts = np.zeros(len(C))
    Ct = []
    # Line 8 : For all transactions T
    for i, t in enumerate(T):
        # Line 9: For all candidate itemsets
        for j, c in enumerate(C):
            #print(c,t)
            # Line 10: If current candidate exists in dataset increase its count
            if set(c).issubset(set(t)):
                #print("In check", c,t)
                Ct.append(c)
                ckCounts[j] += 1
    ckCounts /= numOfTransactions
    #print("C:", C)
    #print("Print Ck counts: ", ckCounts)
    # Line 12: Chek to see which candidates are large for next step
    Lk_1 = [C[i] for i in range(len(C)) if ckCounts[i] >= H[C[i][0]]]
    #print("Lk_1: ", Lk_1)
    # Lk is k x itemsets at each level
    Lk.append(Lk_1)
    k += 1
            
        
print("Lk: ", Lk , "\n")

Iteration:  2
Iteration:  3
Length of candidate:  44
Iteration:  4
Length of candidate:  1
Lk:  [[[4, 5], [4, 6], [4, 8], [4, 9], [4, 10], [4, 3], [4, 2], [6, 8], [6, 9], [6, 10], [6, 1], [7, 9], [7, 20], [7, 50], [8, 9], [8, 10], [9, 10], [9, 20], [9, 50], [9, 2], [10, 30], [10, 40], [10, 50], [10, 2], [20, 50], [30, 40], [30, 50], [40, 50], [50, 3]], [[4, 6, 9], [4, 8, 9], [4, 9, 10], [4, 2, 8], [4, 2, 10], [6, 8, 9], [6, 9, 10], [6, 1, 9], [7, 9, 20], [7, 20, 50], [9, 20, 50], [10, 30, 40], [10, 40, 50], [30, 40, 50]], []] 



In [252]:

ls = [[int(i)] for i in L1]
answer = [ls] + Lk

print(len(answer[0]), answer[0])
print(len(answer[1]), answer[1])
print(len(answer[2]), answer[2])
print(len(answer[3]), answer[3])

13 [[4], [5], [6], [7], [8], [9], [10], [20], [30], [40], [50], [3], [1]]
29 [[4, 5], [4, 6], [4, 8], [4, 9], [4, 10], [4, 3], [4, 2], [6, 8], [6, 9], [6, 10], [6, 1], [7, 9], [7, 20], [7, 50], [8, 9], [8, 10], [9, 10], [9, 20], [9, 50], [9, 2], [10, 30], [10, 40], [10, 50], [10, 2], [20, 50], [30, 40], [30, 50], [40, 50], [50, 3]]
14 [[4, 6, 9], [4, 8, 9], [4, 9, 10], [4, 2, 8], [4, 2, 10], [6, 8, 9], [6, 9, 10], [6, 1, 9], [7, 9, 20], [7, 20, 50], [9, 20, 50], [10, 30, 40], [10, 40, 50], [30, 40, 50]]
0 []


In [261]:
def format_save_string(L, debug = False):
    s = ""
    
    for k, lk in enumerate(L):
        s += '(' + 'Length-' + str(k+1)+' '+ str(len(lk)) +'\n'
        for i, item in enumerate(lk):
            s += 8*' '+"(" + ' '.join(([str(c) for c in item]))+ ')\n'
        s += ')\n'
    if debug: print(s)
    return s

def write_string_to_file(s, wFile = 'results1.txt'):
    
    try:
        with open(wFile, 'w') as wf:
            wf.write(s)
    except Exception as e:
        print(e)
        
s = format_save_string(answer)
write_string_to_file(s)


'\n1 \n2 \n3'